In [25]:
import polars as pl
from datasets import load_dataset, Dataset

# Load the dataset from Huggingface
dataset: Dataset = load_dataset("OpenPipe/hacker-news", split="train")

# Convert the dataset to a Polars DataFrame
df: pl.DataFrame = dataset.to_polars()

del dataset

df

Resolving data files:   0%|          | 0/39 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/39 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/39 [00:00<?, ?it/s]

id,type,by,time,title,text,url,score,parent,top_level_parent,descendants,kids,deleted,dead
i64,str,str,datetime[μs],str,str,str,i64,i64,i64,i64,list[i64],bool,bool
0,null,null,null,null,null,null,null,null,0,null,null,true,null
1,"""story""","""pg""",2006-10-09 18:21:51,"""Y Combinator""",null,"""http://ycombinator.com""",57,null,1,15,"[15, 234509, … 82729]",null,null
2,"""story""","""phyllis""",2006-10-09 18:30:28,"""A Student's Guide to Startups""",null,"""http://www.paulgraham.com/mit.…",16,null,2,0,null,null,null
3,"""story""","""phyllis""",2006-10-09 18:40:33,"""Woz Interview: the early days …",null,"""http://www.foundersatwork.com/…",7,null,3,0,[531602],null,null
4,"""story""","""onebeerdave""",2006-10-09 18:47:42,"""NYC Developer Dilemma""",null,"""http://avc.blogs.com/a_vc/2006…",5,null,4,0,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…
41813380,"""comment""","""slightwinder""",2024-10-11 20:26:18,null,"""&gt; It genuinely astounds me …",null,null,41810691,41808943,null,null,null,null
41813381,"""comment""","""marcosdumay""",2024-10-11 20:26:21,null,"""Add forced sedentarism into th…",null,null,41812891,41811263,null,null,null,null
41813382,"""story""","""turkeynecks""",2024-10-11 20:26:24,"""Mini DOOM-like FPS in BooBoo p…",null,"""https://www.indiedb.com/games/…",1,null,41813382,0,null,null,null


In [26]:
# Ok, I want to start by limiting to comments
comments_df = df.filter(pl.col("type") == "comment")


# add a new column siblings_count
comments_df = comments_df.with_columns(
    [pl.col("id").count().over("parent").alias("siblings_count")]
)
comments_df

id,type,by,time,title,text,url,score,parent,top_level_parent,descendants,kids,deleted,dead,siblings_count
i64,str,str,datetime[μs],str,str,str,i64,i64,i64,i64,list[i64],bool,bool,u32
15,"""comment""","""sama""",2006-10-09 19:51:01,null,"""&#34;the rising star of ventur…",null,null,1,1,null,[17],null,null,4
17,"""comment""","""pg""",2006-10-09 19:52:45,null,"""Is there anywhere to eat on Sa…",null,null,15,1,null,[1079],null,null,1
22,"""comment""","""pg""",2006-10-10 02:18:22,null,"""It's kind of funny that Sevin …",null,null,21,21,null,null,null,null,1
23,"""comment""","""starklysnarky""",2006-10-10 02:30:53,null,"""This is interesting, but the l…",null,null,20,20,null,null,null,null,1
30,"""comment""","""spez""",2006-10-10 15:34:59,null,"""Stay tuned...""",null,null,29,29,null,[31],null,null,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
41813379,"""comment""","""kbolino""",2024-10-11 20:26:09,null,"""A non-blocking send would work…",null,null,41812752,41809262,null,null,null,null,1
41813380,"""comment""","""slightwinder""",2024-10-11 20:26:18,null,"""&gt; It genuinely astounds me …",null,null,41810691,41808943,null,null,null,null,4
41813381,"""comment""","""marcosdumay""",2024-10-11 20:26:21,null,"""Add forced sedentarism into th…",null,null,41812891,41811263,null,null,null,null,1


In [27]:
stories_with_100_points = df.filter(pl.col("score") >= 100)

# filter comments_df to only include comments whose top_level_parent is in the stories_with_100_points dataframe
comments_df = comments_df.filter(
    pl.col("top_level_parent").is_in(stories_with_100_points["id"])
)

comments_df

id,type,by,time,title,text,url,score,parent,top_level_parent,descendants,kids,deleted,dead,siblings_count
i64,str,str,datetime[μs],str,str,str,i64,i64,i64,i64,list[i64],bool,bool,u32
194,"""comment""","""jmzachary""",2007-02-20 22:33:51,null,"""Thanks for the rationale. I'm …",null,null,189,189,null,"[205, 422, 199]",null,null,20
195,"""comment""","""jdroid""",2007-02-20 22:36:52,null,"""You've filled a hole reddit wa…",null,null,189,189,null,[259],null,null,20
199,"""comment""","""Zak""",2007-02-20 22:48:33,null,"""I don't think the fact that th…",null,null,194,189,null,"[1644, 1897]",null,null,3
205,"""comment""","""ninwa""",2007-02-20 23:30:23,null,"""Really? I was most interested …",null,null,194,189,null,"[210, 209]",null,null,3
209,"""comment""","""ninwa""",2007-02-20 23:41:34,null,"""This comment added through the…",null,null,205,189,null,null,null,null,2
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
41813369,"""comment""","""whall6""",2024-10-11 20:25:08,null,"""If you truly only have 5 minut…",null,null,41812596,41811263,null,null,null,null,1
41813371,"""comment""","""julianeon""",2024-10-11 20:25:25,null,"""But consider the tradeoff: it&…",null,null,41811539,41811263,null,null,null,null,13
41813375,"""comment""","""throw0101c""",2024-10-11 20:25:51,null,"""&gt; <i>There is so much coal.…",null,null,41811328,41807681,null,null,null,null,1


In [28]:
sibling_ranks = dict()

# iterate over the rows of the dataframe
for groupable_comment in df.select(pl.col("kids")).iter_rows():
    kids = groupable_comment[0]
    if kids is not None:
        for i, kid in enumerate(kids):
            sibling_ranks[kid] = i + 1

# Find the maximum value in the sibling_ranks dictionary
max_sibling_rank = max(sibling_ranks.values())

# Print the maximum value
print(f"The maximum sibling rank is: {max_sibling_rank}")
print(f"The number of sibling ranks is: {len(sibling_ranks)}")

comments_df = comments_df.with_columns(
    [
        pl.col("id")
        .replace_strict(
            list(sibling_ranks.keys()), list(sibling_ranks.values()), default=-1
        )
        .alias("sibling_rank")
    ]
)
del sibling_ranks

comments_df

The maximum sibling rank is: 1810
The number of sibling ranks is: 36315132


id,type,by,time,title,text,url,score,parent,top_level_parent,descendants,kids,deleted,dead,siblings_count,sibling_rank
i64,str,str,datetime[μs],str,str,str,i64,i64,i64,i64,list[i64],bool,bool,u32,i64
194,"""comment""","""jmzachary""",2007-02-20 22:33:51,null,"""Thanks for the rationale. I'm …",null,null,189,189,null,"[205, 422, 199]",null,null,20,13
195,"""comment""","""jdroid""",2007-02-20 22:36:52,null,"""You've filled a hole reddit wa…",null,null,189,189,null,[259],null,null,20,3
199,"""comment""","""Zak""",2007-02-20 22:48:33,null,"""I don't think the fact that th…",null,null,194,189,null,"[1644, 1897]",null,null,3,3
205,"""comment""","""ninwa""",2007-02-20 23:30:23,null,"""Really? I was most interested …",null,null,194,189,null,"[210, 209]",null,null,3,1
209,"""comment""","""ninwa""",2007-02-20 23:41:34,null,"""This comment added through the…",null,null,205,189,null,null,null,null,2,2
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
41813369,"""comment""","""whall6""",2024-10-11 20:25:08,null,"""If you truly only have 5 minut…",null,null,41812596,41811263,null,null,null,null,1,1
41813371,"""comment""","""julianeon""",2024-10-11 20:25:25,null,"""But consider the tradeoff: it&…",null,null,41811539,41811263,null,null,null,null,13,1
41813375,"""comment""","""throw0101c""",2024-10-11 20:25:51,null,"""&gt; <i>There is so much coal.…",null,null,41811328,41807681,null,null,null,null,1,1


In [34]:
# Let's limit to comments for which all of their parents have a sibling rank <=
# 3. If a comment isn't in the top 3 of its parent it's less likely to have been
# read and upvoted by enough people to form a representative sample.

# We'll iteratively add comments with sibling_rank <= 3 up to 6 iterations.

filtered_comments_df = comments_df.filter(
    (pl.col("sibling_rank") <= 3) & (pl.col("parent").eq(pl.col("top_level_parent")))
)

for iteration in range(6):
    print(f"Iteration {iteration + 1}, current length: {len(filtered_comments_df)}")
    new_filtered_comments_df = comments_df.filter(
        (
            (pl.col("sibling_rank") <= 3)
            & (pl.col("parent").is_in(filtered_comments_df["id"]))
        )
        | (pl.col("id").is_in(filtered_comments_df["id"]))
    )

    if len(new_filtered_comments_df) == len(filtered_comments_df):
        break

    filtered_comments_df = new_filtered_comments_df

comments_df = filtered_comments_df

del filtered_comments_df
comments_df

Iteration 1, current length: 533314
Iteration 2, current length: 1614377


Iteration 3, current length: 2828828
Iteration 4, current length: 3874258
Iteration 5, current length: 4636895
Iteration 6, current length: 5139157


id,type,by,time,title,text,url,score,parent,top_level_parent,descendants,kids,deleted,dead,siblings_count,sibling_rank
i64,str,str,datetime[μs],str,str,str,i64,i64,i64,i64,list[i64],bool,bool,u32,i64
195,"""comment""","""jdroid""",2007-02-20 22:36:52,null,"""You've filled a hole reddit wa…",null,null,189,189,null,[259],null,null,20,3
259,"""comment""","""whatsreal""",2007-02-21 06:18:50,null,"""HaHa! Yes thank you Paul, I w…",null,null,195,189,null,"[453, 7198]",null,null,1,1
287,"""comment""","""ced""",2007-02-21 09:08:16,null,"""Since community-building is pa…",null,null,189,189,null,[365],null,null,20,2
353,"""comment""","""jhenzie""",2007-02-21 18:47:34,null,"""Paul, can we play with arc…",null,null,189,189,null,[7199],null,null,20,1
365,"""comment""","""jdroid""",2007-02-21 19:25:14,null,"""If I bought yspace.com would y…",null,null,287,189,null,null,null,null,1,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
41813352,"""comment""","""dullcrisp""",2024-10-11 20:22:59,null,"""A <i>scrappy</i> marketing tea…",null,null,41813266,41811263,null,null,null,null,1,1
41813354,"""comment""","""ragnese""",2024-10-11 20:23:06,null,"""It was only true that Chrome w…",null,null,41813112,41809698,null,null,null,null,1,1
41813362,"""comment""","""layer8""",2024-10-11 20:23:58,null,"""It’s not simply a limitation i…",null,null,41812523,41812523,null,null,null,null,31,3


In [35]:
# limit all comments to ones that have at least 5 siblings and fewer than 25
# siblings. If there are more than 25 I don't trust the peoples' votes since
# they probably haven't had time to read all of them.
comments_df = comments_df.filter(
    (pl.col("siblings_count") >= 5) & (pl.col("siblings_count") < 25)
)

comments_df

id,type,by,time,title,text,url,score,parent,top_level_parent,descendants,kids,deleted,dead,siblings_count,sibling_rank
i64,str,str,datetime[μs],str,str,str,i64,i64,i64,i64,list[i64],bool,bool,u32,i64
195,"""comment""","""jdroid""",2007-02-20 22:36:52,null,"""You've filled a hole reddit wa…",null,null,189,189,null,[259],null,null,20,3
287,"""comment""","""ced""",2007-02-21 09:08:16,null,"""Since community-building is pa…",null,null,189,189,null,[365],null,null,20,2
353,"""comment""","""jhenzie""",2007-02-21 18:47:34,null,"""Paul, can we play with arc…",null,null,189,189,null,[7199],null,null,20,1
27552,"""comment""","""pg""",2007-06-12 19:02:46,null,"""Don't worry, I'm not going to …",null,null,27550,27550,null,"[28315, 27559, … 28523]",null,null,7,1
27560,"""comment""","""danielha""",2007-06-12 19:15:56,null,"""Prediction: this submission re…",null,null,27550,27550,null,[28098],null,null,7,2
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
41812977,"""comment""","""shadowgovt""",2024-10-11 19:50:42,null,"""Their incentive is really to m…",null,null,41810118,41809698,null,null,null,null,7,-1
41813078,"""comment""","""itronitron""",2024-10-11 19:59:06,null,"""This is the type of comment th…",null,null,41812493,41811263,null,"[41813116, 41813266]",null,null,9,3
41813079,"""comment""","""ragnese""",2024-10-11 19:59:16,null,"""The vast, <i>vast</i>, majorit…",null,null,41809962,41809698,null,null,null,null,12,-1


In [36]:
# filter comments that have non-null text and text length > 10
comments_df = comments_df.filter(
    (pl.col("text").is_not_null())
    & (pl.col("text").str.len_chars() > 10)
    & (pl.col("sibling_rank") > 0)
)

comments_df

id,type,by,time,title,text,url,score,parent,top_level_parent,descendants,kids,deleted,dead,siblings_count,sibling_rank
i64,str,str,datetime[μs],str,str,str,i64,i64,i64,i64,list[i64],bool,bool,u32,i64
195,"""comment""","""jdroid""",2007-02-20 22:36:52,null,"""You've filled a hole reddit wa…",null,null,189,189,null,[259],null,null,20,3
287,"""comment""","""ced""",2007-02-21 09:08:16,null,"""Since community-building is pa…",null,null,189,189,null,[365],null,null,20,2
353,"""comment""","""jhenzie""",2007-02-21 18:47:34,null,"""Paul, can we play with arc…",null,null,189,189,null,[7199],null,null,20,1
27552,"""comment""","""pg""",2007-06-12 19:02:46,null,"""Don't worry, I'm not going to …",null,null,27550,27550,null,"[28315, 27559, … 28523]",null,null,7,1
27560,"""comment""","""danielha""",2007-06-12 19:15:56,null,"""Prediction: this submission re…",null,null,27550,27550,null,[28098],null,null,7,2
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
41812565,"""comment""","""vessenes""",2024-10-11 19:14:29,null,"""Tirzepatide and Semaglutide ar…",null,null,41812493,41811263,null,[41812815],null,null,9,1
41812643,"""comment""","""squidlogic""",2024-10-11 19:23:13,null,"""In my experience fitness is le…",null,null,41812339,41811263,null,"[41812886, 41812696, … 41813092]",null,null,6,1
41812717,"""comment""","""jrflowers""",2024-10-11 19:29:53,null,"""&gt; People in the 50s weren&#…",null,null,41812339,41811263,null,"[41813315, 41812727]",null,null,6,3


In [37]:
from tqdm import tqdm
from typing import List, TypedDict, Dict
from collections import defaultdict
import datetime


progress_bar = tqdm(total=len(comments_df), desc="Processing groups")


class GroupableComment(TypedDict):
    id: int
    parent: int
    time: datetime.datetime
    sibling_rank: int
    top_level_parent: int


class RewardPair(TypedDict):
    chosen: int  # the id of the chosen comment
    rejected: int  # the id of the rejected comment
    chosen_rank: int
    rejected_rank: int
    top_level_parent: int


def process_group(group: List[GroupableComment]) -> list[RewardPair]:
    """
    The goal of this function is to create a reward dataset for Hacker News
    comments. It takes in a dataframe containing all the comments within a given
    hour, and then breaks them down by the parent id.

    Within each parent id group, it orders the comments by sibling rank (that
    is, their relative position in the thread). Lower sibling rank means the
    comment is higher in the thread, which we can assume means it had more
    upvotes and is better.

    We take each comment and its directly following sibling and mark those as
    "chosen" and "rejected", creating a pairwise dataset we can use for reward
    modeling.
    """
    if len(group) == 0:
        return []

    sorted_group = sorted(group, key=lambda x: x["sibling_rank"])

    top_comment = sorted_group[0]
    other_comments = sorted_group[1:]

    group_reward_pairs = []

    for other_comment in other_comments:
        # Make sure the comments were posted within 30 minutes of each other. If
        # they were too far apart, they likely aren't comparable since the older
        # one is likely to win.
        if (other_comment["time"] - top_comment["time"]).total_seconds() > 1800:
            break

        group_reward_pairs.append(
            {
                "chosen": top_comment["id"],
                "rejected": other_comment["id"],
                "chosen_rank": top_comment["sibling_rank"],
                "rejected_rank": other_comment["sibling_rank"],
                "top_level_parent": top_comment["top_level_parent"],
            }
        )

    return group_reward_pairs


reward_pairs: list[RewardPair] = []
parent_group: list[GroupableComment] = []
current_parent = None

sorted_by_parent = comments_df.select(
    pl.col("id"),
    pl.col("sibling_rank"),
    pl.col("parent"),
    pl.col("time"),
    pl.col("top_level_parent"),
).sort("parent")

for comment in sorted_by_parent.iter_rows(named=True):
    comment = GroupableComment(**comment)

    # We already sorted by parent, so if the current comment's parent isn't the
    # same as the previous one, we know we've moved on to a new parent and can
    # process the previous set of comments.
    if comment["parent"] != current_parent:
        reward_pairs.extend(process_group(parent_group))

        parent_group = []
        current_parent = comment["parent"]

    parent_group.append(comment)
    progress_bar.update(1)

# Close the progress bar
progress_bar.close()

pairs_df = pl.DataFrame(reward_pairs)

pairs_df

Processing groups: 100%|██████████| 1104098/1104098 [00:03<00:00, 347241.39it/s]


chosen,rejected,chosen_rank,rejected_rank,top_level_parent
i64,i64,i64,i64,i64
353,287,1,2,189
353,195,1,3,189
27552,27560,1,2,27550
77373,77337,1,2,77246
77666,77438,1,2,77246
…,…,…,…,…
41812258,41812168,1,3,41811263
41812339,41812380,1,2,41811263
41812339,41812405,1,3,41811263


In [43]:
import dicttoxml

sample_pairs = pairs_df.sample(n=20000, seed=42)

progress_bar = tqdm(total=len(sample_pairs) * 2, desc="Building prompts")


def build_prompt(comment_id: int) -> str:
    comment = df.row(comment_id, named=True)
    story = df.row(comment["top_level_parent"], named=True)

    data = {
        "instructions": "Your goal is to analyze the following comment and estimate how highly it will be upvoted by the Hacker News community.",
        "comment": {
            "author": comment["by"],
            "text": comment["text"],
            "parent_chain": [],
        },
        "story": {"title": story["title"]},
    }

    current_parent = df.row(comment["parent"], named=True)
    while current_parent["id"] != story["id"]:
        data["comment"]["parent_chain"].append(
            {"author": current_parent["by"], "text": current_parent["text"]}
        )
        current_parent = df.row(current_parent["parent"], named=True)

    if story["url"] is not None:
        data["story"]["url"] = story["url"]
    if story["text"] is not None:
        data["story"]["text"] = story["text"]

    xml = dicttoxml.dicttoxml(data, attr_type=False, root=False)

    progress_bar.update(1)
    return xml.decode("utf-8")


print(build_prompt(788))

sample_pairs = sample_pairs.with_columns(
    pl.col("chosen")
    .map_elements(build_prompt, return_dtype=pl.String)
    .alias("chosen_prompt"),
    pl.col("rejected")
    .map_elements(build_prompt, return_dtype=pl.String)
    .alias("rejected_prompt"),
)

sample_pairs


Building prompts: 200001it [03:09, 1057.80it/s]


<instructions>Your goal is to analyze the following comment and estimate how highly it will be upvoted by the Hacker News community.</instructions><comment><author>pg</author><text>Harder to say about the upper bound. The factors you mention may play a role, especially kids. I also think need for money is a factor. Most people who could make money have by 40, and once you have enough, you&apos;re less willing to endure the pain of starting a startup.&lt;p&gt;The average age of YC founders is about 25.</text><parent_chain><item><author>jmzachary</author><text>Have you discovered an upper bound? The commonly held belief is that people over X (where 30 &amp;#60; X 35?) can&apos;t start startups because they have kids and a mortgage, or they just aren&apos;t in-tune with what&apos;s hip. Curiously, that excludes people who have &amp;#34;been there and seen that&amp;#34;, the best current example being AJAX. Also, those who have taken a few more trips around the Sun also may recognize niche

chosen,rejected,chosen_rank,rejected_rank,top_level_parent,split,chosen_prompt,rejected_prompt
i64,i64,i64,i64,i64,str,str,str
8880111,8879128,1,2,8876929,"""train""","""<instructions>Your goal is to …","""<instructions>Your goal is to …"
14535547,14533965,1,2,14532751,"""train""","""<instructions>Your goal is to …","""<instructions>Your goal is to …"
14224460,14222513,1,2,14221501,"""train""","""<instructions>Your goal is to …","""<instructions>Your goal is to …"
30012706,30012750,1,3,30011655,"""train""","""<instructions>Your goal is to …","""<instructions>Your goal is to …"
22838384,22837692,1,2,22832914,"""train""","""<instructions>Your goal is to …","""<instructions>Your goal is to …"
…,…,…,…,…,…,…,…
7426463,7426444,1,3,7426327,"""train""","""<instructions>Your goal is to …","""<instructions>Your goal is to …"
34568387,34566820,1,3,34566560,"""train""","""<instructions>Your goal is to …","""<instructions>Your goal is to …"
3958317,3958320,1,2,3958251,"""val""","""<instructions>Your goal is to …","""<instructions>Your goal is to …"


In [47]:
import numpy as np
from datasets import Dataset, DatasetDict

# Get unique top_level_parent values
unique_parents = pairs_df["top_level_parent"].unique()

# Randomly assign splits
np.random.seed(42)  # for reproducibility
split_assignments = np.random.choice(
    ["train", "test", "val"], size=len(unique_parents), p=[0.8, 0.1, 0.1]
)

# Create a dictionary mapping top_level_parent to split
parent_to_split = dict(zip(unique_parents, split_assignments))

# Add the new 'split' column
pairs_df = pairs_df.with_columns(
    pl.col("top_level_parent").replace_strict(parent_to_split).alias("split")
)

# Verify the split proportions
split_counts = pairs_df["split"].value_counts().sort("count", descending=True)
print("Split counts:")
print(split_counts)

# Calculate percentages
total = split_counts["count"].sum()
percentages = (split_counts["count"] / total * 100).round(2)
print("\nSplit percentages:")
print(percentages)


train_df = sample_pairs.filter(pl.col("split") == "train")
validation_df = sample_pairs.filter(pl.col("split") == "val")
test_df = sample_pairs.filter(pl.col("split") == "test")


# Create a DatasetDict to hold all splits
dataset_dict = DatasetDict(
    {
        "train": Dataset.from_polars(train_df.sample(n=10000, seed=42)),
        "validation": Dataset.from_polars(validation_df.sample(n=500, seed=42)),
        "test": Dataset.from_polars(test_df.sample(n=1000, seed=42)),
    }
)

# Save the dataset with splits
dataset_dict.save_to_disk("./data/sample_pairs")

Split counts:
shape: (3, 2)
┌───────┬────────┐
│ split ┆ count  │
│ ---   ┆ ---    │
│ str   ┆ u32    │
╞═══════╪════════╡
│ train ┆ 297780 │
│ test  ┆ 37610  │
│ val   ┆ 36025  │
└───────┴────────┘

Split percentages:
shape: (3,)
Series: 'count' [f64]
[
	80.17
	10.13
	9.7
]


Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]